<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/MultilayerPerceptronClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import tensorflow as tf
import numpy as np

# -------------------------
# 1. Create dummy dataset
# -------------------------
# num_samples = 1000
# num_features = 32
# num_classes = 10

# X = np.random.randn(num_samples, num_features).astype("float32")
# y = np.random.randint(0, num_classes, size=(num_samples,))

# Load MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize to 0–1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Flatten 28x28 → 784
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# -------------------------
# 2. Build your MLP model
# -------------------------
class MLP(tf.keras.Model):
    def __init__(self, layer_size):
        super().__init__()
        self.weights_list = []
        self.bias_list = []

        for in_dim, out_dim in zip(layer_size[:-1], layer_size[1:]):
            w = self.add_weight(
                shape=(in_dim, out_dim),
                initializer=tf.keras.initializers.HeNormal(),
                trainable=True
            )
            b = self.add_weight(
                shape=(out_dim,),
                initializer="zeros",
                trainable=True
            )
            self.weights_list.append(w)
            self.bias_list.append(b)

    def call(self, x):
        out = x
        for w, b in zip(self.weights_list[:-1], self.bias_list[:-1]):
            # out = tf.nn.relu(tf.matmul(out, w) + b)
            out = tf.nn.gelu(tf.matmul(out, w) + b)

        logits = tf.matmul(out, self.weights_list[-1]) + self.bias_list[-1]
        #for multiclass classification.
        return tf.nn.softmax(logits)
        # for binary classification
        # return tf.nn.sigmoid(logits)

# -------------------------
# 3. Instantiate model
# -------------------------
# Corrected: The first element of layer_size must match the input features (784 for MNIST)
model = MLP([784, 512, 512, 256, 128, 64, 10])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# -------------------------
# 4. Train model
# -------------------------
# Corrected: Use x_train and y_train for training
model.fit(x_train, y_train, epochs=5, batch_size=32)

# -------------------------
# 5. Evaluate model
# -------------------------
# Corrected: Use x_test and y_test for evaluation
loss, acc = model.evaluate(x_test, y_test)
print("Test accuracy:", acc)

# -------------------------
# 6. Make predictions
# -------------------------
sample = x_train[:5]
pred = model(sample)
print("Predicted class probabilities:\n", pred.numpy())
print("Predicted classes:", tf.argmax(pred, axis=1).numpy())

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.8981 - loss: 0.3362
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 32s 17ms/step - accuracy: 0.9714 - loss: 0.0942
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9802 - loss: 0.0669
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9855 - loss: 0.0489
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9887 - loss: 0.0374
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9752 - loss: 0.0916
Test accuracy: 0.9796000123023987
Predicted class probabilities:
 [[9.31491861e-09 1.35876085e-11 1.91381147e-11 3.51412200e-05
  2.31152857e-11 9.99962926e-01 1.71912788e-07 8.94537500e-11
  4.12665258e-07 1.26185989e-06]
 [1.00000000e+00 8.64347060e-13 6.42081319e-11 5.83289597e-11
  1.90970243e-12 1.75774256e-10 1.74547647e-08 2.87640842e-13
  6.78307688e-11 2.24842284e-10]
 [6.79571022e-09 4.57912774e-09 9.84930239e-07 4.68095944e-11
  9.99977231e-01 8.50517934e-10 5.9138